#**JSON estructurado con prompts y herramientas**

### **Instalar e importar librerias**

In [ ]:
%pip install openai
%pip install spacy
%pip install openai pydantic

from openai import OpenAI
from google import genai
from google.colab import userdata
import json
import spacy
import re
from datetime import datetime
from pydantic import BaseModel, Field
from typing import List, Literal

### **Acceder a gemini**

In [ ]:
client = OpenAI(
    api_key=userdata.get('GOOGLE_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

texto = "Alberto compró 3 tenis por 5000 pesos el 27 de Junio en Dportenis."

##**Named Entity Recognition (NER)**
**NER** es una tarea fundamental del Procesamiento de Lenguaje Natural (NLP) que consiste en identificar y clasificar "entidades nombradas" en un texto en categorías predefinidas como nombres de personas, organizaciones, ubicaciones, fechas, cantidades, etc.

**Tipos de entidades más comunes**
* Personas
* Organizaciones
* Ubicaciones
* Fechas y horas
* Cantidades
* Productos
* Eventos
* Otros: enfermedades, leyes, obras artísticas, hashtags, etc.

**Ventajas**
* Extraccion precisa
* Multilingüe
* Zero-shot/Few Shot
* Flexible
* Integrable

**Desventajas**
* Ambigüedad
* Dependencia del contexto
* Alucinaciones en LLMs
* No interpretable


**Ejemplo de Entidades:**

Texto: "Alberto compró 3 tenis por 5000 pesos el 27 de Junio en Dportenis."

**Entidades NER:**
```
Alberto: NOMBRE / PERSONA
Tenis: PRODUCTO
3: CANTIDAD
5000: CANTIDAD / DINERO
pesos: MONEDA
27 de Junio: FECHA
Dportenis: TIENDA
```

Una vez identificadas, estas entidades pueden mapearse directamente a un objeto JSON:

```
{
  "nombre": "Alberto",
  "producto": "tenis",
  "cantidad": 3,
  "precio_total": 5000,
  "moneda": "pesos",
  "fecha": "27 de Junio",
  "tienda": "Dportenis"
}
```

In [ ]:
message = [
        {
            "role": "user",
            "content": (
                f"""
               Extrae las entidades nombradas de este texto y clasifícalas como persona, producto, cantidad, cantidad / dinero, moneda, tienda:
              {texto}
              """
            )
        }
    ]

response = client.chat.completions.create(
  model="models/gemini-2.0-flash",
  messages=message)

print(response.choices[0].message.content.strip())

##**Extracción de Relaciones**
La Extracción de Relaciones va un paso más allá de NER. No solo identifica entidades, sino que también determina las relaciones semánticas entre ellas. Es decir, cómo se conectan las entidades entre sí en el texto.

**Ejemplo de Relaciones:**

Texto: "Alberto compró 3 tenis por 5000 pesos el 27 de Junio en Dportenis."

**Relaciones:**
```
Alberto, compró, 3 tenis
compró, por, 5000 pesos
compró, el, 27 de Junio
compró, en, Dportenis
```

Una vez identificadas, estas relaciones pueden mapearse directamente a un objeto JSON:

```
{
  "nombre": "Alberto",
  "tienda": "Dportenis"
  "compra: {
    "producto": "tenis",
    "cantidad": 3,
    "precio_total": 5000,
    "moneda": "pesos",
    "fecha": "27 de Junio"
  }
}
```

In [ ]:
message = [
        {
            "role": "user",
            "content": (
                f"""
               Extrae las relaciones de este texto:
              {texto}
              """
            )
        }
    ]

response = client.chat.completions.create(
  model="models/gemini-2.0-flash",
  messages=message)

print(response.choices[0].message.content.strip())

##**Prompting para Control Estructural**
Esta es la técnica fundamental para guiar a los LLMs a generar JSON. Implica diseñar prompts que le digan al modelo exactamente qué estructura de JSON debe producir, incluyendo los nombres de las claves, los tipos de datos esperados para cada valor, y cualquier anidamiento.

In [ ]:
message = [
        {
            "role": "user",
            "content": (
                f"""
               Extrae la información del siguiente texto y devuélvela en formato JSON siguiendo la estructura indicada. Asegúrate de respetar los tipos de datos (número, cadena de texto, fecha):

                Texto:
                {texto}

                Formato JSON esperado:
                {{
                  "cliente": "string",
                  "producto": "string",
                  "cantidad": "number",
                  "precio_total": "number",
                  "moneda": "string",
                  "fecha": "string (YYYY-MM-DD)",
                  "lugar": "string"
                }}
              """
            )
        }
    ]

response = client.chat.completions.create(
  model="models/gemini-2.0-flash",
  messages=message)

print(response.choices[0].message.content.strip())

##**[JSON API](https://jsonapi.org/)**

**JSON:API** es una especificación para construir APIs RESTful que utilizan JSON como formato de intercambio.
Define reglas estrictas sobre cómo estructurar datos, cómo representar relaciones y cómo manejar errores, con el objetivo de garantizar consistencia, eficiencia y compatibilidad entre aplicaciones.

Ejemplo:
```
{
  "data": {
    "type": "productos",
    "id": "001",
    "attributes": {
      "nombre": "tenis",
      "precio": 1500
    }
  }
}
```

In [ ]:
message = [
        {
            "role": "user",
            "content": (
                f"""
               Extrae la información del siguiente texto y devuélvela en formato JSON:API bien definido siguiendo la estructura indicada, incluye relationships e included.

                Texto:
                {texto}
              """
            )
        }
    ]

response = client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=message,
    response_format={
      "type": "json_object"
    }
)

print(response.choices[0].message.content.strip())

## **json_mode**
Se refiere a activar o forzar al modelo para que toda su salida sea parsable como JSON, sin texto adicional o errores de formato que impedirían su lectura por un programa.

Hay varias maneras de lograr el json_mode:

* Mediante un system prompt claro y explícito
* Mediante el parámetro ***response_format***
* Mediante el parámetro ***response_format*** + ***strict***
* Mediante el parámetro ***json schema***

### **response_format=json**
Es una de las maneras más robustas de asegurarte de que un LLM te devuelva datos estructurados.

In [ ]:

message = [
        {
            "role": "user",
            "content": (
                f"""
               Extrae la información del siguiente texto y devuélvela en formato JSON siguiendo la estructura indicada. Asegúrate de respetar los tipos de datos (número, cadena de texto, fecha):

                Texto:
                {texto}

                Formato JSON esperado:
                {{
                  "cliente": "string",
                  "producto": "string",
                  "cantidad": "number",
                  "precio_total": "number",
                  "moneda": "string",
                  "fecha": "string (YYYY-MM-DD)",
                  "lugar": "string"
                }}
              """
            )
        }
    ]

response = client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=message,
    response_format={
      "type": "json_object"
    },
    temperature=0.0
)

print(response.choices[0].message.content.strip())



### **Strict**

**strict** es una opción que se puede activar cuando usas **response_format="json"**.
Obliga al modelo a responder con un JSON válido y exacto, sin explicaciones ni texto adicional. Esto garantiza que la salida sea parseable directamente, ideal para integraciones automáticas.

### **Schema**
Schema es una especificación estándar que te permite describir y validar la estructura, el formato y las restricciones de cualquier documento JSON. Su propósito es asegurar que los datos JSON cumplen con un conjunto predefinido de reglas.

In [ ]:
class DetallesProducto(BaseModel):
    """Detalles de un producto adquirido."""
    nombre: str = Field(
        description="Nombre o descripción del producto adquirido (por ejemplo: 'tenis', 'calcetas')."
    )
    cantidad: int = Field(
        description="Cantidad de unidades de este producto."
    )

class InformacionCompra(BaseModel):
    """Información estructurada de una transacción de compra en Dportenis."""
    cliente: str = Field(
        description="Nombre completo del cliente que realizó la compra."
    )
    productos: List[DetallesProducto] = Field(
        description="Lista de productos individuales comprados."
    )
    precio_total: float = Field(
        description="Precio total de la transacción."
    )
    moneda: Literal["MXN", "USD", "EUR"] = Field(
        description="Moneda de la transacción. Solo se permite 'MXN', 'USD' o 'EUR'."
    )
    fecha_compra: str = Field(
        description="Fecha de la compra en formato AAAA-MM-DD (por ejemplo: 2024-06-27)."
    )
    tienda: str = Field(
        description="Sucursal o lugar de Dportenis donde se realizó la compra."
    )


message = [
        {
            "role": "user",
            "content": (
                f"""
                 Texto de la compra: {texto}
              """
            )
        }
    ]

response = client.beta.chat.completions.parse(
    model="gemini-2.0-flash",
    messages=message,
    response_format=InformacionCompra,
    temperature=0.0
)

print(response.choices[0].message.content.strip())


## **spaCy vs LLMs**



### **spaCy**

**spaCy** es una librería de código abierto en Python para el procesamiento avanzado del lenguaje natural. Está diseñada para ser rápida, eficiente y lista para producción.

**Características Clave**:
* Modelos Pre-entrenados
* Enfoque Supervisado
* Eficiencia

**Ventajas**:
* Rendimiento y Velocidad
* Costo
* Control y Determinismo
* Precisión en Dominio Específico
* Localidad

**Desventajas**:
* Flexibilidad Limitada
* Generalización Pobre
* No Comprende Intenciones Complejas

In [ ]:
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    spacy.cli.download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

doc = nlp(texto)

compra_info = {
    "cliente": None,
    "producto": None,
    "cantidad": None,
    "precio_total": None,
    "moneda": None,
    "fecha": None,
    "lugar": None
}

for ent in doc.ents:
    if ent.label_ == "PER":
        compra_info["cliente"] = ent.text
    elif ent.label_ == "ORG":
        compra_info["lugar"] = ent.text
    elif ent.label_ in ["LOC", "GPE"]:
        compra_info["lugar"] = compra_info["lugar"] or ent.text
    elif ent.label_ == "DATE":
        try:
            compra_info["fecha"] = datetime.strptime(ent.text.strip(), "%d de %B").replace(year=2024).strftime("%Y-%m-%d")
        except:
            compra_info["fecha"] = ent.text

cantidad_match = re.search(r"(\d+)\s+(\w+)", texto)
if cantidad_match:
    compra_info["cantidad"] = int(cantidad_match.group(1))
    compra_info["producto"] = cantidad_match.group(2)

precio_match = re.search(r"(\d+)\s*(pesos|€|dólares|USD|EUR)?", texto, re.IGNORECASE)
if precio_match:
    compra_info["precio_total"] = int(precio_match.group(1))
    if precio_match.group(2):
        compra_info["moneda"] = precio_match.group(2).lower()

print(f"Texto original: {texto}\n")
print("JSON estructurado:")
print(json.dumps(compra_info, indent=2, ensure_ascii=False))


### **LLMs**

Son modelos de redes neuronales masivamente grandes, pre-entrenados en cantidades gigantescas de texto de internet. Su fortaleza radica en su capacidad para entender, generar y razonar sobre el lenguaje natural de una manera muy flexible, a menudo con poca o ninguna "instrucción" específica.

**Características Clave**:
* Generativos
* Pre-entrenados y Adaptables
* Comprensión Contextual

**Ventajas**:
* Flexibilidad y Adaptabilidad
* Extracción de Relaciones Complejas
* Manejo de Variaciones Lingüísticas
* JSON Directo

**Desventajas**:
* Costo
* Latencia
* "Alucinaciones"
* Privacidad de Datos

In [ ]:
message = [
        {
            "role": "user",
            "content": (
                f"""
               Extrae la información del siguiente texto y devuélvela en formato JSON válido con la siguiente estructura:
                  {{
                    "cliente": "string",
                    "producto": "string",
                    "cantidad": "number",
                    "precio_total": "number",
                    "moneda": "string",
                    "fecha": "string (YYYY-MM-DD)",
                    "lugar": "string"
                  }}
                  Texto:
                  {texto}
              """
            )
        }
    ]

response = client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=message,
    response_format={
      "type": "json_object"
    },
)

print(response.choices[0].message.content.strip())

## **Ejercicio 03**

Analiza el siguiente texto y con el ejercicio de ejemplo de LLMs (visto previamente) genera una salida estructurada tipo JSON utilizando la estructura de tu preferencia:

Texto: Mariana Torres realizó 2 compras en diferentes sucursales de Dportenis durante el mes de mayo de 2024.

Su primera compra fue el 2 de mayo en la sucursal de Guadalajara, donde adquirió:
* 1 par de tenis Adidas Ultraboost por $4200 pesos

* 1 camiseta Dry-Fit por $850 pesos

Pagó con tarjeta de crédito y utilizó el cupón DPORT20, que le dio un 20% de descuento en el segundo artículo.
El número de orden fue 001245, y recibió el correo de confirmación en mariana.torres@example.com. Eligió envío estándar (5-7 días).

El 10 de mayo, Mariana visitó la sucursal de Monterrey, donde compró:
* 2 pares de calcetas Nike por $600 pesos

* 1 mochila Puma por $1200 pesos

Esta compra la realizó en efectivo. No utilizó cupón. El número de orden fue 001322 y solicitó recolección en tienda.

In [ ]:
texto = "Mariana Torres realizó 2 compras en diferentes sucursales de Dportenis durante el mes de mayo de 2024. Su primera compra fue el 2 de mayo en la sucursal de Guadalajara, donde adquirió: 1 par de tenis Adidas Ultraboost por $4200 pesos, 1 camiseta Dry-Fit por $850 pesos. Pagó con tarjeta de crédito y utilizó el cupón DPORT20, que le dio un 20% de descuento en el segundo artículo. El número de orden fue 001245, y recibió el correo de confirmación en mariana.torres@example.com. Eligió envío estándar (5-7 días). El 10 de mayo, Mariana visitó la sucursal de Monterrey, donde compró: 2 pares de calcetas Nike por $600 pesos, 1 mochila Puma por $1200 pesos. Esta compra la realizó en efectivo. No utilizó cupón. El número de orden fue 001322 y solicitó recolección en tienda."

message = [

    ]

response = client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=message,
)

print(response.choices[0].message.content.strip())